<a href="https://colab.research.google.com/github/gauravshetty98/Gaurav-GIS-Repo/blob/main/data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Extraction and Cleaning

This notebook represents the steps to download the zip file from GitHub repository and extract the data from its content and create a single dataset.

Dataset :  https://www.samhsa.gov/data/report/2021-nsduh-state-specific-tables

The dataset contains state-wise PDFs with drug abuse and mental health information of the respective state.

We start of by installing `tabula` library. This library is used to extract the tables from the PDFs.


In [1]:
!pip install -q tabula-py

Once the library is installed, we move on to import all the required libraries. We make use of google drive to store the PDFs present in the dataset.

**This notebook requires your google drive mounting to store dataset files. You can delete those files after running the notebook**



In [2]:
import os
from os import path
import tabula
import pandas as pd
import numpy as np

from google.colab import drive

drive.mount("/content//gdrive")

Drive already mounted at /content//gdrive; to attempt to forcibly remount, call drive.mount("/content//gdrive", force_remount=True).


### Creating folders for storing all the data

`2021NSDUHsaeSpecificStatesTabs122022` is the name of the new folder. This folder will contain all the state wise PDFs.
A `tables` folder is also present, which will contain all the tables extracted from each state PDF.



In [3]:
dir = "//content//gdrive//My Drive//2021NSDUHsaeSpecificStatesTabs122022"
folder_name = "tables"

Creating a new folder if it is not present

In [4]:
if os.path.exists(dir) == False:
    os.mkdir(dir)
    print("done")
if os.path.exists(dir+"//"+folder_name) == False:
    os.mkdir(dir+"//"+folder_name)
    print("done")

!ls "//content//gdrive//My Drive//2021NSDUHsaeSpecificStatesTabs122022"

done
done
tables


Downloading the zip file from GitHub and extracting it into the drive folder we created

In [5]:
!wget "https://github.com/gauravshetty98/Gaurav-GIS-Repo/raw/main/2021NSDUHsaeSpecificStatesTabs122022.zip"
!unzip /content/2021NSDUHsaeSpecificStatesTabs122022.zip -d "//content//gdrive//My Drive//2021NSDUHsaeSpecificStatesTabs122022"

--2023-09-14 23:59:51--  https://github.com/gauravshetty98/Gaurav-GIS-Repo/raw/main/2021NSDUHsaeSpecificStatesTabs122022.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gauravshetty98/Gaurav-GIS-Repo/main/2021NSDUHsaeSpecificStatesTabs122022.zip [following]
--2023-09-14 23:59:51--  https://raw.githubusercontent.com/gauravshetty98/Gaurav-GIS-Repo/main/2021NSDUHsaeSpecificStatesTabs122022.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13301032 (13M) [application/zip]
Saving to: ‘2021NSDUHsaeSpecificStatesTabs122022.zip.5’

2021NSDUHsaeSpecifi 100%[===================>]  12.68M  --.-KB/s  

### Concatenating all files and creating a single DataFrame

We iterate through all PDF files present in the new directory and extract the tables present in the first two pages using `tabula.read_pdf()`

We then concatenate the 2 extracted tables into a single dataframe and add a new column `states` to the dataframe.

Finally, we convert the dataframe into csv and store it in the `tables` folder in google drive

In [6]:
ext = ('.pdf','.PDF')
final_table = pd.DataFrame()
for files in os.listdir(dir):
    if files.endswith(ext):
        print(files)
        tables = tabula.read_pdf(dir + "//" + files,pages=[1,2]) #address of pdf file
        filename = files.replace("NSDUHsae","").replace(".pdf","").replace("2021","")
        df1 = pd.DataFrame(tables[0])
        df2 = pd.DataFrame(tables[1])
        final_table = pd.concat([df1,df2])
        final_table['states'] = filename
        #print(final_table)
        final_table.to_csv(os.path.join(dir + "//" + folder_name, filename+".csv"), index=False)
        print(filename)
    else:
        print("Else: ", files)
#final_table.to_csv(os.path.join(folder_name, "FinalTable.csv"), index=False)

Else:  tables
Else:  images
NSDUHsaeAlabama2021.pdf
Alabama
NSDUHsaeAlaska2021.pdf
Alaska
NSDUHsaeArizona2021.pdf
Arizona
NSDUHsaeArkansas2021.pdf
Arkansas
NSDUHsaeCalifornia2021.pdf
California
NSDUHsaeColorado2021.pdf
Colorado
NSDUHsaeConnecticut2021.pdf
Connecticut
NSDUHsaeDelaware2021.pdf
Delaware
NSDUHsaeDistrictOfCol2021.pdf
DistrictOfCol
NSDUHsaeFlorida2021.pdf
Florida
NSDUHsaeGeorgia2021.pdf
Georgia
NSDUHsaeHawaii2021.pdf
Hawaii
NSDUHsaeIdaho2021.pdf
Idaho
NSDUHsaeIllinois2021.pdf
Illinois
NSDUHsaeIndiana2021.pdf
Indiana
NSDUHsaeIowa2021.pdf
Iowa
NSDUHsaeKansas2021.pdf
Kansas
NSDUHsaeKentucky2021.pdf
Kentucky
NSDUHsaeLouisiana2021.pdf
Louisiana
NSDUHsaeMaine2021.pdf
Maine
NSDUHsaeMaryland2021.pdf
Maryland
NSDUHsaeMassachusetts2021.pdf
Massachusetts
NSDUHsaeMichigan2021.pdf
Michigan
NSDUHsaeMidwest2021.pdf
Midwest
NSDUHsaeMinnesota2021.pdf
Minnesota
NSDUHsaeMississippi2021.pdf
Mississippi
NSDUHsaeMissouri2021.pdf
Missouri
NSDUHsaeMontana2021.pdf
Montana
NSDUHsaeNational2021.pdf
N

Output shows all the CSVs extracted and stored state wise in the tables folder

In [7]:
!ls "//content//gdrive//My Drive//2021NSDUHsaeSpecificStatesTabs122022/tables"

Alabama.csv	   Indiana.csv	      National.csv	 RhodeIsland.csv
Alaska.csv	   Iowa.csv	      Nebraska.csv	 SouthCarolina.csv
Arizona.csv	   Kansas.csv	      Nevada.csv	 South.csv
Arkansas.csv	   Kentucky.csv       NewHampshire.csv	 SouthDakota.csv
California.csv	   Louisiana.csv      NewJersey.csv	 Tennessee.csv
Colorado.csv	   Maine.csv	      NewMexico.csv	 Texas.csv
Connecticut.csv    Maryland.csv       NewYork.csv	 Utah.csv
Delaware.csv	   Massachusetts.csv  NorthCarolina.csv  Vermont.csv
DistrictOfCol.csv  Michigan.csv       NorthDakota.csv	 Virginia.csv
Florida.csv	   Midwest.csv	      Northeast.csv	 Washington.csv
Georgia.csv	   Minnesota.csv      Ohio.csv		 West.csv
Hawaii.csv	   Mississippi.csv    Oklahoma.csv	 WestVirginia.csv
Idaho.csv	   Missouri.csv       Oregon.csv	 Wisconsin.csv
Illinois.csv	   Montana.csv	      Pennsylvania.csv	 Wyoming.csv


All the CSVs are concatenated into a single dataframe.

In [8]:
final_dataset = pd.DataFrame()
csv_path = dir + "//" + folder_name
for tables in os.listdir(csv_path):
  df3 = pd.read_csv(csv_path + "//" + tables)
  final_dataset = pd.concat([final_dataset,df3],ignore_index = True)

---------------------
## Data Cleaning

Here we start with the data cleaning. We first replace all the `NaN` values with 0.

In [9]:
print(final_dataset.head(10))

                                             Measure    12+ 12-17 18-25  \
0                                      ILLICIT DRUGS    NaN   NaN   NaN   
1              Illicit Drug Use in the Past Month1,2    403    17    98   
2                     Marijuana Use in the Past Year    537    30   152   
3                    Marijuana Use in the Past Month    315    16    90   
4  Perceptions of Great Risk from Smoking Marijua...  1,128   105    83   
5  First Use of Marijuana in the Past Year among ...    NaN   NaN   NaN   
6                                             Use3,4     40    15    20   
7  Illicit Drug Use Other Than Marijuana in the P...    132     7    20   
8                       Cocaine Use in the Past Year     67     1    14   
9  Perceptions of Great Risk from Using Cocaine O...  3,066   210   334   

     26+    18+   states  
0    NaN    NaN  Alabama  
1    288    386  Alabama  
2    355    507  Alabama  
3    209    299  Alabama  
4    940  1,023  Alabama  
5    NaN    

### Dropping Empty Rows

The dataset contains some rows which are empty and dont contain any necessary information. Here we search for those rows and drop them from the dataset

Count of empty rows in the dataset = 392

In [9]:
from zmq import NULL
count = 0
for i in range(0,final_dataset.shape[0]):
  if list(final_dataset.iloc[i,1:6]) == list([np.NaN, np.NaN, np.NaN, np.NaN, np.NaN]):
    final_dataset.iloc[i+1,0] = str(final_dataset.iloc[i,0]) + " " + str(final_dataset.iloc[i+1,0])
    count = count + 1

print("Number of empty rows: ",count)
print("Acutal data we need: ", final_dataset.shape[0] - count)

Number of empty rows:  392
Acutal data we need:  2016


We make use of the `dropna()` function to drop all the empty rows. You can see in the resulting dataframe there are no empty rows.

Final shape of the dataset = 2016 * 7

In [10]:
final_dataset = final_dataset.dropna()
final_dataset = final_dataset.reset_index()
print(final_dataset.head(10))

   index                                            Measure    12+ 12-17  \
0      1  ILLICIT DRUGS Illicit Drug Use in the Past Mon...    403    17   
1      2                     Marijuana Use in the Past Year    537    30   
2      3                    Marijuana Use in the Past Month    315    16   
3      4  Perceptions of Great Risk from Smoking Marijua...  1,128   105   
4      6  First Use of Marijuana in the Past Year among ...     40    15   
5      7  Illicit Drug Use Other Than Marijuana in the P...    132     7   
6      8                       Cocaine Use in the Past Year     67     1   
7      9  Perceptions of Great Risk from Using Cocaine O...  3,066   210   
8     10                       Heroin Use in the Past Year5     --    --   
9     11  Perceptions of Great Risk from Trying Heroin O...  3,567   236   

  18-25    26+    18+   states  
0    98    288    386  Alabama  
1   152    355    507  Alabama  
2    90    209    299  Alabama  
3    83    940  1,023  Alabama 

### Replacing more Null values

The dataset still contains some null values. During the extraction process the null values in the PDF were converted to string `"--"`. Also there are some `*` in the dataset which represent data with low precision or no estimates. We search for these strings/values and replace it with zero as they are of no use in our dataset.

In [15]:
print("Before: ", final_dataset.iloc[260,:])
final_dataset = final_dataset.replace("--",0)
final_dataset = final_dataset.replace("*",0)
print("After: ", final_dataset.iloc[260,:])

Before:  index                               311
Measure    Heroin Use in the Past Year5
12+                                  --
12-17                                --
18-25                                 *
26+                                   7
18+                                   9
states                         Delaware
Name: 260, dtype: object
After:  index                               311
Measure    Heroin Use in the Past Year5
12+                                   0
12-17                                 0
18-25                                 0
26+                                   7
18+                                   9
states                         Delaware
Name: 260, dtype: object


--------
### Storing the dataset into a CSV file

In [16]:
final_dataset.to_csv(os.path.join(dir + "//" + folder_name, "state_dataset.csv"), index=False)

------
The Cleaning of NSDUH dataset is done. This dataset will be further used for the plotting problem sets.

This notebook will be updated and new sections will be added if any new dataset is being imported and it requires some cleaning